### 연웅

In [ ]:
import moviepy.editor
import os
import random
import fnmatch
import math

base_directory = r'D:\2022-불법영상물-유튜브'

ext = "*mp4"
length = 30 # 30초
short_edge = 256

labels = ["agree", "nonagree"]

for label in labels:
    directory = os.path.join(base_directory, label)
    dst_directory = os.path.join(base_directory + "-after", label)

    if not os.path.exists(dst_directory):
        os.makedirs(dst_directory)

    # compile list of videos
    inputs = [os.path.join(directory,f) for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f)) and fnmatch.fnmatch(f, ext)]

    for idx, name in enumerate(inputs):
        orig_file_name = os.path.basename(name)

        # import to moviepy
        clip = moviepy.editor.VideoFileClip(name)

        # 강제로 256x256으로 만듬. 비율 달라지는건 상관 x
        clip = clip.resize((short_edge, short_edge))

        clip_start = []

        for i in range (0, math.floor(clip.duration)):
            if i % 30 == 0 and i + 30 < clip.duration:
                clip_start.append(i)
        
        clip_start = random.sample(clip_start, 5)

        clips = []

        for start in clip_start:
            # cut a subclip
            clips.append(clip.subclip(start,start+length))

        for idx, c in enumerate(clips):
            c.write_videofile(os.path.join(base_directory + "-after", label, orig_file_name.split("_")[0] + "-" + f'{idx+1:03d}.mp4'))

### 현택

In [ ]:
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
import cv2
import random

# original_path = 'F:/video_dataset/nsrvideo/train/legal/[#유미의세포들] 박이 터졌다는 건 이별을 준비해야 한다는 것💔 유자청 사건부터 단답 문자까지 참고 참다가 결국 터져버린 유미🤯  #Diggle #갓구운클립.mp4'
original_path = 'F:/video_dataset/nsrvideo/val/illegal/'
partition_path = 'F:/video_dataset/nsrvideo-partition/val/illegal/'

video_names = os.listdir(original_path)

for name in video_names:
    try:
        video = cv2.VideoCapture(original_path + name)
        total_fps = video.get(cv2.CAP_PROP_FRAME_COUNT)
        fps = video.get(cv2.CAP_PROP_FPS)

        times = []

        start_time = 0
        end_minute = (total_fps // fps)
        # end_second = int((total_fps / fps) % 60)

        count = int((total_fps / fps) // 60 // 5)

        if end_minute >= 0.5:
            for i in range(count):
                times.append(start_time)
                start_time += 30
        else:
            times.append(start_time)

        if len(times) >= 5:
            times = random.sample(times, 5)

        for i, time in enumerate(times):
            ffmpeg_extract_subclip(original_path + name, start_time+1, start_time+30, targetname=partition_path + f'/{name[:-4]}-{i}.mp4')
    except:
        continue

# ffmpeg_extract_subclip(original_path, 60, 70, targetname=partition_path)